In [12]:
# Process each class folder
for label, class_dir in enumerate(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, class_dir)
    
    # Check if the path is a directory
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image = Image.open(img_path).convert('L')  # Convert to grayscale
            image = image.resize((64, 64))  # Resize to 64x64
            image = np.array(image) / 255.0  # Normalize to [0, 1]
            data.append(image)
            labels.append(label)

In [13]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

# Path to your extracted dataset
dataset_dir = "C:/Users/naomi/Downloads/asl_alphabet_test"

# Lists to hold data and labels
data = []
labels = []

# Process each image file and infer the label from the filename
for img_name in os.listdir(dataset_dir):
    img_path = os.path.join(dataset_dir, img_name)
    
    # Ensure only image files are processed
    if os.path.isfile(img_path) and img_name.endswith(('.png', '.jpg', '.jpeg')):
        # Extract label from filename (e.g., "A" from "A_test.jpg")
        label = ord(img_name[0].upper()) - 65  # Convert 'A' to 0, 'B' to 1, ..., 'Z' to 25
        labels.append(label)
        
        # Load image, convert to grayscale, resize, normalize, and add to data
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        image = image.resize((64, 64))  # Resize to 64x64
        image = np.array(image) / 255.0  # Normalize to [0, 1]
        data.append(image)

# Convert to numpy arrays and reshape
data = np.array(data).reshape(-1, 64, 64, 1)  # Reshape to (samples, 64, 64, 1)
labels = np.array(labels)
labels = to_categorical(labels, num_classes=26)  # Assuming 26 classes (A-Z)

# Check the shapes of data and labels
print(data.shape)  # Expected: (num_samples, 64, 64, 1)
print(labels.shape)  # Expected: (num_samples, 26)


(28, 64, 64, 1)
(28, 26)


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(26, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=10
)
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
# Save the model
model.save('sign_language_model.keras')


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 3.2599 - val_accuracy: 0.0000e+00 - val_loss: 3.3773
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.0909 - loss: 3.1847 - val_accuracy: 0.0000e+00 - val_loss: 3.6229
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.0909 - loss: 3.1254 - val_accuracy: 0.0000e+00 - val_loss: 4.0724
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.1364 - loss: 3.1846 - val_accuracy: 0.0000e+00 - val_loss: 4.2960
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.0909 - loss: 3.2375 - val_accuracy: 0.0000e+00 - val_loss: 4.2200
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.1364 - loss: 3.2768 - val_accuracy: 0.0000e+00 - val_loss: 4.0073
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.1364 - loss: 3.1450 - val_accuracy: 0.0000e+00 - val_loss: 3.8227
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.1364 - loss: 3.0935 - val